# CBHAR averages
This notebook creates climate normals from the Chukchi Beaufort High-resolution Atmospheric Reanalysis (CBHAR) 1979-2009. More information about these data can be found at: http://portal.aoos.org/#module-metadata/9c1581d0-8674-11e6-9dd3-00265529168c/1eadaf41-e4ca-49e5-aa0e-c2bdbe889560

Climate normals should be calculated as:
- 30 years of data
- Calculate monthly averages (e.g., Jan 2000, Jan 2001, Jan 2002)
- Take the mean across all years

In [ ]:
# Import existing code modules
%matplotlib inline
import time
import datetime
import numpy as np
from numpy import ma
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
from matplotlib import ticker
import netCDF4
from tqdm import trange
import nbconvert

In [ ]:
# load data from the public URL
ncd = netCDF4.Dataset("http://thredds.aoos.org/thredds/dodsC/WRFSFC.nc")

# Get available times
time_var = ncd.variables['TIME']
times=netCDF4.num2date(np.array(time_var), time_var.units)

# Get size of arrays
ny = ncd.dimensions['NY'].size
nx = ncd.dimensions['NX'].size

# Get X and Y values
xval = np.array(ncd.variables['NX'])
yval = np.array(ncd.variables['NY'])

In [ ]:
# Define a function to get a range of data and generate a mean over time
def mean_cbhar(start_day, end_day):
    # Subset the times by the time range given
    start_tindex = np.searchsorted(times, start_day, side='left')
    end_tindex = np.searchsorted(times, end_day, side='right')
    ntimes=end_tindex-start_tindex

    data_var = ncd.variables['T2_C']

    # Predefine the output array
    data = np.zeros((ntimes, ny, nx))
    for i in trange(ntimes, desc=start_day.strftime('%B %Y'), leave=False):
        # this gets one time-slice at a time
        data[i,:,:]=np.array(data_var[start_tindex+i,:,:])

    # Find the mean (ignore math warnings)
    np.seterr(invalid='ignore')
    
    # Return the mean over the input time period
    return np.mean(data, axis=0)

In [ ]:
# range(start, stop) where stop is not included
# Set specific months or years here
# months = [1,7]
# years = [1979, 1980]

# Find monthly averages over all years
months = range(1, 12+1) # stop value is not inclusive
years = range(1979, 2009+1)

nmonths = len(months)
nyears = len(years)
nallmonths=(nmonths * nyears)

In [ ]:
monthly_means = np.zeros((nallmonths, ny, nx))

for month in months:
    
    for year in years:
        start_day = datetime.datetime(year,month,1,0,0)
        
        #Calculate end_day by going to next month and then back one second
        if months[i] < 12:
            end_day = datetime.datetime(year, month+1, 1, 0, 0) - datetime.timedelta(seconds=1)
        else: # if month is 12 we can't go to 13
            end_day = datetime.datetime(year, month, 31, 23, 59) - datetime.timedelta(seconds=1)
            
        # take the mean
        monthly_means[i,:,:] = np.expand_dims(mean_cbhar(start_day, end_day), axis=0)

In [ ]:
# # in case you gotta walk away
np.save('quicksave.sav', monthly_means)
# monthly_means = np.load('quicksave.sav.npy')

In [ ]:
# Define a plotting function
def create_plot(lon, lat, datamask, vmin, vmax, titletext):

    fig = plt.figure(num=None, figsize=(10,7.5))
    ax = plt.subplot(111)
    # remove frames
    ax.spines["top"].set_visible(False)    
    ax.spines["bottom"].set_visible(False)    
    ax.spines["right"].set_visible(False)    
    ax.spines["left"].set_visible(False)
    # only show ticks on bottom and left
    ax.get_xaxis().tick_bottom()    
    ax.get_yaxis().tick_left()

    # set limits
    plt.xlim(min(lon), max(lon))
    plt.ylim(min(lat), max(lat))
    cm.plasma.set_bad(color='grey', alpha=1.) # Set masked values to dark gray
#    plt.title(titletext)
    img = plt.pcolormesh(lon, lat, datamask, vmin=vmin, vmax=vmax, cmap = 'jet')
    
    #Add pretty colorbar
    cax=fig.add_axes([0.25, 0.02, 0.5, 0.03]) #[left, bottom, width, height]
    cb = plt.colorbar(img, cax=cax, orientation='horizontal', extend='both')
    cb.locator = ticker.MaxNLocator(nbins=4)
    cb.update_ticks()
    plt.text(0.39, -2, 'degrees Celsius')

    plt.show()

    print('min: ' + str(np.min(datamask)))
    print('max: ' + str(np.max(datamask)))

In [ ]:
create_plot(xval, yval, monthly_means[0], -40, 20, 'CBHAR Average Temp at 2 meters during June')
create_plot(xval, yval, monthly_means[1], -40, 20, 'CBHAR Average Temp at 2 meters during June')

In [ ]:
# Create bounds (months x 2)
np.array(['1979-1-1','2009-1-31'],
         ['1979-2-1', '2009-3']
        )


In [ ]:
# Write out a netCDF
ncfile = netCDF4.Dataset('CBHAR-test-mean.nc', 'w')

# Add dimensions
ncfile.createDimension('TIME', nmonths)
ncfile.createDimension('NX', nx)
ncfile.createDimension('NY', ny)
ncfile.createDimension('NV', 2) # because we give two values to define each bound

# Add time variable
nc_times=ncfile.createVariable('TIME', float, dimensions=('TIME'))
nc_times.climatology = "climatology_bounds"
nc_times.units = "days since 0-1-1"
nc_times[:] = netCDF4.date2num(datetime.datetime(1979,6,1,0,0), time_var.units)

# Add nx variable
nc_lats=ncfile.createVariable('NX', float, dimensions=('NX'))
nc_lats.point_spacing = 'even'
nc_lats.axis = 'X'
nc_lats[:]=xval

# Add ny variable
nc_lats=ncfile.createVariable('NY', float, dimensions=('NY'))
nc_lats.point_spacing = 'even'
nc_lats.axis = 'Y'
nc_lats[:]=yval

# Add time bounds variable (http://cfconventions.org/cf-conventions/v1.6.0/cf-conventions.html)
nc_bounds=ncfile.createVariable('climatology_bounds', float, dimensions=('TIME', 'NV'))


# Add temperature
# Write
nc_temps = ncfile.createVariable('T2_C', float, dimensions=('TIME', 'NY', 'NX'))
nc_temps.long_name = '2-meter temperature'
nc_temps.cell_methos='time: mean over years'
nc_temps.units = 'C'
nc_temps[:, :,:] = monthly_means

# Add Attributes
ncfile.title = 'CBHAR data average'
ncfile.description = description + 'An average of CBHAR data. Original data and more information about this dataset can be found at http://portal.aoos.org/#module-metadata/9c1581d0-8674-11e6-9dd3-00265529168c/1eadaf41-e4ca-49e5-aa0e-c2bdbe889560'

ncfile.close()